In [1]:
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for saving plots
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from typing import List, Tuple
import random
import os


class Robot:
    def __init__(self, position: Tuple[float, float], direction: Tuple[float, float], robot_id: int):
        self.position = np.array(position, dtype=float)
        self.direction = np.array(direction, dtype=float)
        self.direction = self.direction / np.linalg.norm(self.direction)  # normalize
        self.id = robot_id
        
        # Physical properties
        self.radius = 0.3
        self.speed = 0.05
        
        # Behavioral properties
        self.communication_range = 2.5
        self.avoidance_radius_illuminated = 0.51
        self.avoidance_radius_shadowed = 0.4
        
        # State
        self.is_illuminated = False
        self.current_avoidance_radius = self.avoidance_radius_shadowed
    
    def update_illumination(self, beacon_position: np.ndarray, other_robots: List['Robot']):
        """Check if robot is illuminated by beacon or shadowed by other robots
        
        The beacon emits parallel light rays (like the sun). A robot is illuminated
        unless another robot casts a shadow on it by being between it and the beacon.
        """
        # Assume illuminated by default (beacon shines everywhere)
        self.is_illuminated = True
        
        # Check if any other robot is blocking the light
        for other in other_robots:
            if other.id == self.id:
                continue
            
            # Vector from beacon to this robot
            to_this = self.position - beacon_position
            # Vector from beacon to other robot
            to_other = other.position - beacon_position
            
            # Check if other robot is between beacon and this robot
            # For parallel rays (like sun), we check if other robot is closer to beacon
            # and within the shadow cone
            
            distance_this = np.linalg.norm(to_this)
            distance_other = np.linalg.norm(to_other)
            
            if distance_other >= distance_this:
                continue  # Other robot is farther or at same distance, can't cast shadow
            
            # Check if this robot is in the shadow cone of the other robot
            # Direction from beacon (normalized)
            if distance_this > 0:
                direction = to_this / distance_this
            else:
                continue
            
            # Project other robot's position onto the ray to this robot
            projection = np.dot(to_other, direction)
            
            # Find the closest point on the ray to the other robot's center
            closest_point_on_ray = beacon_position + direction * projection
            
            # Distance from other robot's center to the ray
            distance_to_ray = np.linalg.norm(other.position - closest_point_on_ray)
            
            # If other robot intersects the light ray (within its radius), it casts a shadow
            if distance_to_ray < other.radius and 0 < projection < distance_this:
                self.is_illuminated = False
                break
        
        # Update avoidance radius based on illumination
        if self.is_illuminated:
            self.current_avoidance_radius = self.avoidance_radius_illuminated
        else:
            self.current_avoidance_radius = self.avoidance_radius_shadowed
    
    def check_neighbors(self, other_robots: List['Robot']) -> int:
        """Count robots within communication range"""
        count = 0
        for other in other_robots:
            if other.id == self.id:
                continue
            distance = np.linalg.norm(self.position - other.position)
            if distance <= self.communication_range:
                count += 1
        return count
    
    def detect_obstacle(self, other_robots: List['Robot']) -> Tuple[bool, np.ndarray]:
        """Check if there's an obstacle within avoidance radius. Returns (has_obstacle, obstacle_position)"""
        for other in other_robots:
            if other.id == self.id:
                continue
            distance = np.linalg.norm(self.position - other.position)
            if distance <= self.current_avoidance_radius:
                return True, other.position
        return False, None
    
    def avoid_obstacle(self, obstacle_position: np.ndarray):
        """Turn away from detected obstacle"""
        # Calculate direction away from obstacle
        away_vector = self.position - obstacle_position
        if np.linalg.norm(away_vector) > 0:
            self.direction = away_vector / np.linalg.norm(away_vector)
    
    def coherence_behavior(self, turn_random: bool = False):
        """Turn 180 degrees to rejoin swarm, or turn randomly if specified"""
        if turn_random:
            # Random angle between 0 and 2π
            angle = random.uniform(0, 2 * np.pi)
            self.direction = np.array([np.cos(angle), np.sin(angle)])
        else:
            # Turn 180 degrees
            self.direction = -self.direction
    
    def move(self):
        """Move forward in current direction"""
        self.position += self.direction * self.speed


class Beacon:
    def __init__(self, position: Tuple[float, float]):
        self.position = np.array(position, dtype=float)
    
    def is_robot_illuminated(self, robot: Robot, all_robots: List[Robot]) -> bool:
        """Check if a robot is illuminated or shadowed by other robots"""
        # This functionality is implemented in Robot.update_illumination
        # but kept here for potential alternative implementation
        pass


class Simulation:
    def __init__(self, num_robots: int, max_steps: int, world_size: Tuple[float, float],
                 initialization_mode: str = 'connected', enable_plotting: bool = False,
                 plot_interval: int = 5, plot_output_dir: str = 'plots'):
        """
        Initialize simulation with specified parameters.
        
        Args:
            num_robots: Number of robots in the swarm
            max_steps: Maximum number of simulation steps
            world_size: Tuple of (width, height) for the world
            initialization_mode: 'connected' (clustered start) or 'random' (random positions)
            enable_plotting: Whether to generate matplotlib plots during simulation
            plot_interval: Record robot positions every N steps for plotting
            plot_output_dir: Directory to save plots (only used if enable_plotting=True)
        """
        self.num_robots = num_robots
        self.max_steps = max_steps
        self.world_size = world_size
        self.initialization_mode = initialization_mode
        self.enable_plotting = enable_plotting
        self.plot_interval = plot_interval
        self.plot_output_dir = plot_output_dir
        self.current_step = 0
        
        self.robots: List[Robot] = []
        self.beacon = None
        
        # Store robot positions at intervals for plotting
        self.position_history = []  # List of (step, robot_positions, robot_states)
        
        self._initialize_robots()
        self._initialize_beacon()
        
        # Create plot output directory if plotting is enabled
        if self.enable_plotting:
            os.makedirs(self.plot_output_dir, exist_ok=True)
            print(f"\n✓ Plot output directory created: '{self.plot_output_dir}/'")

    
    def _initialize_robots(self):
        """Create robots based on initialization mode"""
        if self.initialization_mode == 'connected':
            self._initialize_robots_connected()
        elif self.initialization_mode == 'random':
            self._initialize_robots_random()
        else:
            raise ValueError(f"Unknown initialization mode: {self.initialization_mode}")
        
        # Verify connectivity if in connected mode
        if self.initialization_mode == 'connected':
            self._verify_connectivity()
    
    def _initialize_robots_connected(self):
        """Create robots in a cluster to ensure full connectivity"""
        # Place all robots on the left side of the world, moving generally rightward
        # This simulates a herd migrating toward the beacon (like buffalo toward sun)
        center_x = self.world_size[0] * 0.2  # Start at 20% from left edge
        center_y = self.world_size[1] / 2    # Vertically centered
        
        # Maximum radius to ensure connectivity
        max_radius = 1.0
        
        for i in range(self.num_robots):
            # Random position within the cluster
            angle = random.uniform(0, 2 * np.pi)
            radius = random.uniform(0, max_radius)
            
            x = center_x + radius * np.cos(angle)
            y = center_y + radius * np.sin(angle)
            
            # Direction: generally rightward (toward beacon) with some randomness
            # Random angle within ±45 degrees of rightward (0 radians)
            dir_angle = random.uniform(-np.pi/4, np.pi/4)
            direction = (np.cos(dir_angle), np.sin(dir_angle))
            
            robot = Robot((x, y), direction, i)
            self.robots.append(robot)
    
    def _initialize_robots_random(self):
        """Create robots with random positions across the world"""
        for i in range(self.num_robots):
            # Random position in the world
            x = random.uniform(0, self.world_size[0])
            y = random.uniform(0, self.world_size[1])
            
            # Random direction
            angle = random.uniform(0, 2 * np.pi)
            direction = (np.cos(angle), np.sin(angle))
            
            robot = Robot((x, y), direction, i)
            self.robots.append(robot)
    
    def _verify_connectivity(self):
        """Verify that all robots are connected"""
        print("\nVerifying connectivity...")
        for i, robot in enumerate(self.robots):
            neighbors = robot.check_neighbors(self.robots)
            print(f"  Robot {i}: {neighbors} neighbors")
        
        # Check if all robots have at least N-1 neighbors (fully connected)
        min_neighbors = min(robot.check_neighbors(self.robots) for robot in self.robots)
        if min_neighbors == self.num_robots - 1:
            print("✓ All robots are fully connected!")
        else:
            print(f"⚠ Warning: Minimum neighbors = {min_neighbors}, expected {self.num_robots - 1}")
    
    def _initialize_beacon(self, position: Tuple[float, float] = None):
        """Place beacon in the world"""
        if position is None:
            # Place beacon above and to the right (like the sun in the sky)
            # At the top-right area of the world
            position = (self.world_size[0] * 0.85, self.world_size[1] * 0.85)
        self.beacon = Beacon(position)
    
    def step(self):
        """Execute one simulation time step"""
        # 1. Update illumination status for all robots
        for robot in self.robots:
            robot.update_illumination(self.beacon.position, self.robots)
        
        # 2. Check for obstacles and execute avoidance behavior
        for robot in self.robots:
            has_obstacle, obstacle_pos = robot.detect_obstacle(self.robots)
            if has_obstacle:
                robot.avoid_obstacle(obstacle_pos)
                continue  # Skip coherence check if avoiding obstacle
        
        # 3. Check coherence behavior (only if not avoiding obstacle)
        for robot in self.robots:
            num_neighbors = robot.check_neighbors(self.robots)
            if num_neighbors < self.num_robots - 1:  # α = N means all N-1 other robots
                robot.coherence_behavior(turn_random=False)
        
        # 4. Move all robots
        for robot in self.robots:
            robot.move()
        
        self.current_step += 1
    
    def run(self, visualizer=None):
        """Run the full simulation"""
        print("\n" + "=" * 60)
        print("Running simulation...")
        print("=" * 60 + "\n")
        
        # Record initial state if plotting is enabled
        if self.enable_plotting:
            self._record_state(0)
        
        for step in range(1, self.max_steps + 1):
            self.step()
            
            # Record state at specified intervals
            if self.enable_plotting and step % self.plot_interval == 0:
                self._record_state(step)
            
            # Progress indicator
            if step % 100 == 0:
                print(f"  Completed step {step}/{self.max_steps}")
            
        
        print("\n" + "=" * 60)
        print("Simulation complete!")
        
        # Generate the final plot with all time steps
        if self.enable_plotting:
            print(f"Generating plot with {len(self.position_history)} time steps...")
            self._plot_all_states()
            print(f"Plot saved in '{self.plot_output_dir}/' directory")
        
        print("=" * 60)
        
        if visualizer:
            visualizer.close()
    
    def _record_state(self, step: int):
        """Record the current state of all robots for later plotting"""
        robot_data = []
        for robot in self.robots:
            robot_data.append({
                'id': robot.id,
                'position': robot.position.copy(),
                'direction': robot.direction.copy(),
                'is_illuminated': robot.is_illuminated,
                'avoidance_radius': robot.current_avoidance_radius
            })
        self.position_history.append((step, robot_data))
    
    def _plot_all_states(self):
        """Create a single plot showing robot positions at all recorded time steps"""
        # Create a figure with subplots - one for each time step
        num_plots = len(self.position_history)
        cols = 5  # 5 plots per row
        rows = (num_plots + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
        if rows == 1:
            axes = axes.reshape(1, -1)
        axes = axes.flatten()
        
        # Plot each time step
        for idx, (step, robot_data) in enumerate(self.position_history):
            ax = axes[idx]
            
            # Set up the subplot
            ax.set_xlim(0, self.world_size[0])
            ax.set_ylim(0, self.world_size[1])
            ax.set_aspect('equal')
            ax.grid(True, alpha=0.3)
            ax.set_title(f'Step {step}', fontsize=10, fontweight='bold')
            
            # Draw beacon
            beacon_circle = Circle(self.beacon.position, 0.3, color='yellow', 
                                  ec='orange', linewidth=2, zorder=5)
            ax.add_patch(beacon_circle)
            
            # Draw robots
            for robot_state in robot_data:
                if robot_state['is_illuminated']:
                    color = 'green'
                    alpha = 0.1
                else:
                    color = 'red'
                    alpha = 0.1
                
                # Draw avoidance radius
                avoidance_circle = Circle(robot_state['position'], 
                                         robot_state['avoidance_radius'], 
                                         color=color, alpha=alpha, zorder=1)
                ax.add_patch(avoidance_circle)
                
                # Draw robot body
                robot_circle = Circle(robot_state['position'], 0.3, 
                                    color=color, ec='black', linewidth=1.5, 
                                    alpha=0.7, zorder=3)
                ax.add_patch(robot_circle)
                
                # Draw direction arrow
                arrow_length = 0.6
                dx = robot_state['direction'][0] * arrow_length
                dy = robot_state['direction'][1] * arrow_length
                ax.arrow(robot_state['position'][0], robot_state['position'][1], 
                        dx, dy, head_width=0.2, head_length=0.15, 
                        fc='black', ec='black', zorder=4, linewidth=1.5)
        
        # Hide unused subplots
        for idx in range(num_plots, len(axes)):
            axes[idx].axis('off')
        
        # Add legend at the top
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor='green', alpha=0.7, label='Illuminated'),
            Patch(facecolor='red', alpha=0.7, label='Shadowed'),
            plt.Line2D([0], [0], marker='*', color='orange', label='Beacon', 
                      markersize=15, linestyle='None')
        ]
        fig.legend(handles=legend_elements, loc='upper center', 
                  ncol=3, fontsize=12, frameon=True)
        
        plt.suptitle(f'Emergent Taxis Simulation - N={self.num_robots}, Steps=0-{self.max_steps}, Interval={self.plot_interval}', 
                    fontsize=14, fontweight='bold', y=0.995)
        plt.tight_layout(rect=[0, 0, 1, 0.99])
        
        # Save plot
        filename = os.path.join(self.plot_output_dir, f'emergent_taxis_simulation.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight')
        plt.close()
        
        print(f"  ✓ Saved: {filename}")




# a)

Implement emergent taxis and execute a run with N = 10 robots and 1,000 time steps. Every
5 time steps, plot the positions of all robots in 2D space. Show the illumination status and
avoidance radius for each robot in the same plot.
You are free to use a programming language of your choice, but please include a guide how
to execute your code. You may model the beacon as an area light source with parallel rays of
light. Make sure that all robots are initialized fully connected.

In [2]:
import os


def main():
    """Main function for Part A - Emergent Taxis Simulation"""
    
    print("=" * 60)
    print("EMERGENT TAXIS SIMULATION - PART A")
    print("=" * 60)
    print("Parameters:")
    print("  - Number of robots (N): 10")
    print("  - Maximum steps: 1,000")
    print("  - Plot interval: 5 steps")
    print("  - World size: 20 × 20")
    print("  - Initialization: Fully connected")
    print("=" * 60)
    
    # Create and run simulation with plotting enabled
    sim = Simulation(
        num_robots=10,
        max_steps=1000,
        world_size=(20, 20),
        initialization_mode='connected',
        enable_plotting=True,
        plot_interval=5,
        plot_output_dir='plots'
    )
    
    # Display initial configuration
    print("\nInitial robot positions:")
    for i, robot in enumerate(sim.robots):
        print(f"  Robot {i}: pos=({robot.position[0]:.2f}, {robot.position[1]:.2f}), "
              f"dir=({robot.direction[0]:.2f}, {robot.direction[1]:.2f})")
    
    print(f"\nBeacon position: ({sim.beacon.position[0]:.2f}, {sim.beacon.position[1]:.2f})")
    
    # Check if plotting is enabled
    if sim.enable_plotting:
        print(f"\nPlotting is ENABLED")
        print(f"  Output directory: {sim.plot_output_dir}")
        print(f"  Plot interval: every {sim.plot_interval} steps")
    else:
        print("\nPlotting is DISABLED")
    
    # Run the simulation
    sim.run()
    
    # Verify plots were created
    if sim.enable_plotting and os.path.exists(sim.plot_output_dir):
        plot_files = [f for f in os.listdir(sim.plot_output_dir) if f.endswith('.png')]
        print(f"\nGenerated {len(plot_files)} plot files in '{sim.plot_output_dir}/'")
        if len(plot_files) > 0:
            print(f"  First plot: {plot_files[0]}")
            print(f"  Last plot: {plot_files[-1]}")
    else:
        print(f"\nWarning: No plots found in '{sim.plot_output_dir}/'")


if __name__ == "__main__":
    main()

EMERGENT TAXIS SIMULATION - PART A
Parameters:
  - Number of robots (N): 10
  - Maximum steps: 1,000
  - Plot interval: 5 steps
  - World size: 20 × 20
  - Initialization: Fully connected

Verifying connectivity...
  Robot 0: 9 neighbors
  Robot 1: 9 neighbors
  Robot 2: 9 neighbors
  Robot 3: 9 neighbors
  Robot 4: 9 neighbors
  Robot 5: 9 neighbors
  Robot 6: 9 neighbors
  Robot 7: 9 neighbors
  Robot 8: 9 neighbors
  Robot 9: 9 neighbors
✓ All robots are fully connected!

✓ Plot output directory created: 'plots/'

Initial robot positions:
  Robot 0: pos=(3.97, 9.94), dir=(0.80, 0.60)
  Robot 1: pos=(4.90, 9.70), dir=(0.96, -0.30)
  Robot 2: pos=(4.17, 10.09), dir=(0.76, 0.65)
  Robot 3: pos=(3.83, 10.20), dir=(0.72, 0.70)
  Robot 4: pos=(4.05, 10.03), dir=(0.99, 0.13)
  Robot 5: pos=(4.70, 10.42), dir=(0.93, 0.37)
  Robot 6: pos=(4.04, 9.98), dir=(0.97, 0.25)
  Robot 7: pos=(3.35, 10.54), dir=(1.00, -0.05)
  Robot 8: pos=(4.14, 10.02), dir=(0.91, 0.41)
  Robot 9: pos=(4.73, 10.51), 